In [1]:
import json
from os.path import isfile

statement_section_lst=dict()
with open('../data/Gold_test.json','r') as f:
    train_json=json.load(f)
    for k in train_json:
        line=train_json[k]
        premises=json.load(open('../data/Training_data/CT json/{}.json'.format(line['Primary_id'])))
        statement_section_lst[k]=[
            line['Statement'],
            json.dumps(premises[line['Section_id']]),
            line['Label'],
            "Neither",
            line['Section_id'],
            line['Type']
        ]
        if 'Secondary_id' in line:
            premises=json.load(open('../data/Training_data/CT json/{}.json'.format(line['Secondary_id'])))
            statement_section_lst[k][1]=json.dumps(
                json.loads(statement_section_lst[k][1])+premises[line['Section_id']]
            )

In [2]:
len(statement_section_lst)

500

In [3]:
with open('../data/Categories-test/Inference type/Numerical.json','r') as f:
    for k in json.load(f):
        statement_section_lst[k][3]='Numerical'
        
with open('../data/Categories-test/Inference type/NLI.json','r') as f:
    for k in json.load(f):
        statement_section_lst[k][3]='NLI'

In [4]:
import pandas as pd
import numpy as np

test_df=pd.DataFrame([statement_section_lst[k] for k in statement_section_lst],columns=['statement','section','label','InferenceType','SectionType','Type'])
test_df['section']=test_df['section'].apply(lambda x:json.dumps([s.strip() for s in json.loads(x)]))
test_df

,statement,section,label,InferenceType,SectionType,Type
0,Women suffering from both claustrophobia and I...,"[""Inclusion Criteria:"", ""Women are eligible to...",Contradiction,NLI,Eligibility,Comparison
1,There are no conditions on mental mental healt...,"[""Inclusion Criteria:"", ""Women 18 years"", ""His...",Entailment,NLI,Eligibility,Single
2,the secondary trial requires patients to be ov...,"[""Inclusion Criteria:"", ""Patients presenting f...",Entailment,NLI,Eligibility,Comparison
3,Patients currently taking part in the secondar...,"[""Inclusion Criteria:"", ""Be a female of any ra...",Entailment,NLI,Eligibility,Comparison
4,HER2 + Patients in the primary trial receive t...,"[""INTERVENTION 1:"", ""Trastuzumab"", ""Participan...",Contradiction,Numerical,Intervention,Comparison
...,...,...,...,...,...,...
495,alcoholic patients are excluded from the prima...,"[""Inclusion Criteria:"", ""All participants will...",Contradiction,Numerical,Eligibility,Comparison
496,Patients with a histologically/cytologically c...,"[""Inclusion Criteria:"", ""Voluntarily signed an...",Entailment,NLI,Eligibility,Single
497,the primary trial treats one of its patient co...,"[""INTERVENTION 1:"", ""Pregabalin150"", ""Patients...",Entailment,Numerical,Intervention,Comparison
498,after a month of TAK-228 Plus Tamoxifen treatm...,"[""Outcome Measurement:"", ""Ki67 Expression"", ""K...",Contradiction,Numerical,Results,Single


In [5]:
test_df.to_csv('./output/categorized_test_statement_section.csv',index=False)